In [52]:
import numpy
from PIL import Image, ImageDraw
import json
import matplotlib.pyplot as plt 
import cv2

In [128]:
# dir_ = '0-531/' #до 348 размечено 
# dir_ = '1064-1595/' #до 01324 размечено
# dir_ = '1596-2127(возможно кривойjson)/' #да кривой, оба джейсона дают все пустые маски

# dir_ = '2128-2659/' #до 2427 размечено
# dir_ = '2660-3191/' # до 2965

dir_ = '3192-3723/' # 3292

# dir_ = '532-1063/' #как-то неравномерно 

In [129]:
json_name = "MRI Lumbar Part2.json"
# json_name = 'MRI Lumbar Part2.json.json'
# json_name = 'MRI_2659.json'
# json_name = 'MRI_3191.json'
# беру всегда part2, когда он есть, потому что они выглядят the same


In [130]:
raw_data = []
with open(dir_+json_name, "r") as file: 
    raw_data = json.load(file)

In [131]:
all_keys = []
for key in raw_data['_via_img_metadata'].keys():
    all_keys.append(key)
len(all_keys)

532

In [132]:
pics_path = dir_ + 'pictures/'
masks_path = dir_ + 'masks/'

In [133]:
import os
if not os.path.exists(masks_path):
    os.makedirs(masks_path)

In [134]:
fucked_imgs = []
zero_files = []
for k in range(0, len(all_keys)):
    filename = raw_data['_via_img_metadata'][all_keys[k]]['filename']

    image = Image.open(pics_path + filename)
    width = image.size[0] 
    height = image.size[1]
    
#     img = Image.new('LA', (width, height), 0)
    img1 = Image.new('L',(width, height), 0)
    img2 = Image.new('L',(width, height), 0)
    img_old = Image.new('L',(width, height), 0)
    for i in range(len(raw_data['_via_img_metadata'][all_keys[k]]['regions'])):
        try:
            xs = raw_data['_via_img_metadata'][all_keys[k]]['regions'][i]['shape_attributes']['all_points_x']
            ys = raw_data['_via_img_metadata'][all_keys[k]]['regions'][i]['shape_attributes']['all_points_y']
            all_pairs = []
            for j in range(len(ys)):
                all_pairs.append((xs[j], ys[j]))

            if raw_data['_via_img_metadata'][all_keys[k]]['regions'][i]['region_attributes']['А3'] == 'disc':
                ImageDraw.Draw(img1).polygon(all_pairs, outline=1, fill=1)
            elif raw_data['_via_img_metadata'][all_keys[k]]['regions'][i]['region_attributes']['А3'] == 'spinal canal':
                ImageDraw.Draw(img2).polygon(all_pairs, outline=1, fill=1)

            if raw_data['_via_img_metadata'][all_keys[k]]['regions'][i]['region_attributes']['А3'] == 'disc':
                ImageDraw.Draw(img_old).polygon(all_pairs, outline=1, fill=1)
            elif raw_data['_via_img_metadata'][all_keys[k]]['regions'][i]['region_attributes']['А3'] == 'spinal canal':
                ImageDraw.Draw(img_old).polygon(all_pairs, outline=1, fill=1)
        except:
#             print('fucked image - not labeled at all or прямоугольник')
#             print(pics_path + filename)
            fucked_imgs.append(pics_path + filename)
            
        
#     mask = numpy.array(img)
    mask_old = numpy.array(img_old)
    mask1 = numpy.array(img1)
    mask2 = numpy.array(img2)
    
    if len(numpy.unique(mask_old)) == 1:
        zero_files.append(pics_path + filename)
    
    mask = numpy.dstack((mask1, mask2, mask_old))

    #plt.imsave(masks_path +filename, mask)
    cv2.imwrite(masks_path +filename, mask)

In [135]:
len(zero_files)

433

In [136]:
len(fucked_imgs)

0

In [137]:
zero_files[0]

'3192-3723/pictures/03242.PNG'

In [138]:
zero_files[0].split('/')[-2]

'pictures'

In [144]:
# fucked_imgs[0]

# remove to special folders not labeled imgs and their empty masks

In [145]:
zero_masks = []
for file in zero_files:
    parts = file.split('/')
    parts[-2] = 'masks'
    file = '/'.join(parts)
    zero_masks.append(file)

In [146]:
zero_masks[0]

'3192-3723/masks/03242.PNG'

In [147]:
# Create directory
dirName = dir_[:-1]+'_zero'
try:
# Create target Directory
    os.mkdir(dirName)
    print("Directory " , dirName ,  " Created ") 
except FileExistsError:
    print("Directory " , dirName ,  " already exists")
    
try:
# Create target Directory
    os.mkdir(dirName+'/masks')
    print("Directory " , dirName+'/masks' ,  " Created ") 
except FileExistsError:
    print("Directory " , dirName+'/masks' ,  " already exists")
    
try:
# Create target Directory
    os.mkdir(dirName+'/pictures')
    print("Directory " , dirName+'/pictures' ,  " Created ") 
except FileExistsError:
    print("Directory " , dirName+'/pictures' ,  " already exists")

Directory  3192-3723_zero  already exists
Directory  3192-3723_zero/masks  already exists
Directory  3192-3723_zero/pictures  already exists


In [148]:
new_dest_imgs = []
new_dest_masks = []

for file in zero_files:
    parts = file.split('/')
    parts[0] = dirName
    res = '/'.join(parts)
    new_dest_imgs.append(res)
    
for file in zero_masks:
    parts = file.split('/')
    parts[0] = dirName
    res = '/'.join(parts)
    new_dest_masks.append(res)

In [150]:
for i in range(len(zero_files)):
    os.rename(zero_files[i], new_dest_imgs[i])
    os.rename(zero_masks[i], new_dest_masks[i])

# copy non zero imgs and masks to train folder

In [151]:
dirName = 'train_data'
try:
# Create target Directory
    os.mkdir(dirName)
    print("Directory " , dirName ,  " Created ") 
except FileExistsError:
    print("Directory " , dirName ,  " already exists")
    

dirName = 'test_data'
try:
# Create target Directory
    os.mkdir(dirName)
    print("Directory " , dirName ,  " Created ") 
except FileExistsError:
    print("Directory " , dirName ,  " already exists")

Directory  train_data  Created 
Directory  test_data  Created 


In [154]:
dirName = 'train_data/masks'
try:
# Create target Directory
    os.mkdir(dirName)
    print("Directory " , dirName ,  " Created ") 
except FileExistsError:
    print("Directory " , dirName ,  " already exists")
    

dirName = 'train_data/imgs'
try:
# Create target Directory
    os.mkdir(dirName)
    print("Directory " , dirName ,  " Created ") 
except FileExistsError:
    print("Directory " , dirName ,  " already exists")

Directory  train_data/masks  Created 
Directory  train_data/imgs  Created 


In [183]:
dirName = 'test_data/masks'
try:
# Create target Directory
    os.mkdir(dirName)
    print("Directory " , dirName ,  " Created ") 
except FileExistsError:
    print("Directory " , dirName ,  " already exists")
    

dirName = 'test_data/imgs'
try:
# Create target Directory
    os.mkdir(dirName)
    print("Directory " , dirName ,  " Created ") 
except FileExistsError:
    print("Directory " , dirName ,  " already exists")

Directory  test_data/masks  Created 
Directory  test_data/imgs  Created 


In [161]:
list_of_dirs = [
    '0-531/',
    '1064-1595/',
    '2128-2659/',
    '2660-3191/',
    '3192-3723/',
    '532-1063/'
]


# test

In [169]:
# os.listdir(list_of_dirs[0]+'masks') 
# os.listdir(list_of_dirs[0]+'pictures') 

In [171]:
os.listdir(list_of_dirs[0]+'pictures/')[0]

'00006.PNG'

In [172]:
img_list = os.listdir(list_of_dirs[0]+'pictures/') 
masks_list = os.listdir(list_of_dirs[0]+'masks/') 

img_src_pathes = []
for img in img_list:
    img_src_pathes.append(list_of_dirs[0]+'pictures/'+img)
    
mask_src_pathes = []
for mask in masks_list:
    mask_src_pathes.append(list_of_dirs[0]+'masks/'+mask)

In [174]:
# mask_src_pathes

In [178]:
# mask_src_pathes[0]

'0-531/masks/00006.PNG'

In [184]:
# os.listdir('test_data')

['masks', 'imgs']

In [185]:
img_dst_pathes = []
mask_dst_pathes = []

for mask in masks_list:
    mask_dst_pathes.append('test_data/masks/' + mask)
for img in img_list:
    img_dst_pathes.append('test_data/imgs/'+img)

In [188]:
# mask_dst_pathes[10], mask_src_pathes[10]

('test_data/masks/00156.PNG', '0-531/masks/00156.PNG')

In [189]:
for i in range(len(img_dst_pathes)):
    os.rename(img_src_pathes[i], img_dst_pathes[i])
    os.rename(mask_src_pathes[i], mask_dst_pathes[i])

# train data

In [190]:
list_of_dirs[1:]

['1064-1595/', '2128-2659/', '2660-3191/', '3192-3723/', '532-1063/']

In [192]:
for dir_ in list_of_dirs[1:]:
    img_list = os.listdir(dir_+'pictures/') 
    masks_list = os.listdir(dir_+'masks/') 

    img_src_pathes = []
    for img in img_list:
        img_src_pathes.append(dir_+'pictures/'+img)

    mask_src_pathes = []
    for mask in masks_list:
        mask_src_pathes.append(dir_+'masks/'+mask)

    img_dst_pathes = []
    mask_dst_pathes = []

    for mask in masks_list:
        mask_dst_pathes.append('train_data/masks/' + mask)
    for img in img_list:
        img_dst_pathes.append('train_data/imgs/'+img)

    for i in range(len(img_dst_pathes)):
        os.rename(img_src_pathes[i], img_dst_pathes[i])
        os.rename(mask_src_pathes[i], mask_dst_pathes[i])